# Imports

## Preparatory Imports

In [2]:
import sys
import os

src_path = "../src"
sys.path.insert(0, src_path)

hf_token = os.environ["HUGGINGFACE_TOKEN"]

## Main Imports

In [4]:
from data import load_data
from client import Client
from tester import Tester
import pickle

In [ ]:
with open("../data/runs/run2.pickle", "rb") as conversation_file:
    conversations = pickle.load(conversation_file)

# Test the Prototype

In [6]:
data = [load_data(f"../data/processed/train{i}.json") for i in range(1, 4)]
entries = data[0] | data[1]
c = Client(
    "mistralai/Mistral-Nemo-Instruct-2407",
    token=hf_token,
)
t = Tester(c, entries)
t.conversations = conversations
indices = sorted(list(entries.keys()))

In [44]:
N = 84

In [22]:
t.run(indices[N:])

Found an error processing entry 27, question 4. Skipping to the next one...
Found an error processing entry 27, question 5. Skipping to the next one...
Found an error processing entry 30, question 2. Skipping to the next one...
Found an error processing entry 30, question 3. Skipping to the next one...
Found an error processing entry 30, question 4. Skipping to the next one...
Found an error processing entry 30, question 5. Skipping to the next one...
Found an error processing entry 30, question 6. Skipping to the next one...
Found an error processing entry 36, question 1. Skipping to the next one...
Found an error processing entry 49, question 1. Skipping to the next one...
Found an error processing entry 49, question 2. Skipping to the next one...
Found an error processing entry 49, question 3. Skipping to the next one...
Found an error processing entry 66, question 2. Skipping to the next one...
Found an error processing entry 66, question 3. Skipping to the next one...
Found an err

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-Nemo-Instruct-2407/v1/chat/completions (Request ID: gclOrIDydqw4DHcDrfaqX)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [48]:
with open("../data/runs/run2.pickle", "wb") as out:
    pickle.dump(t.conversations, out)